# **(초기)3개창고최적화재고증가반영_ChatGPT요청문.txt**
- 저자가 작성한 요청문으로 생성한 파이썬 코드 BIP 모델
- 재고증가 반영: 창고 운영비에 SQRT(창고개수) 추가
- 입력데이터 (기존) NW_MasterDataV02.xlsx에서 NW_MasterData.xlsx로 파일명 변경
- (출력) 수요박스 개수 컬럼 추가

In [ ]:
!pip install pulp openpyxl

In [ ]:
import pandas as pd
from pulp import LpProblem, LpVariable, LpMinimize, lpSum, LpBinary
from openpyxl import Workbook
import math

# 파일 경로 설정
input_file_path = '/content/sample_data/NW_MasterData.xlsx'
output_file_path = '/content/sample_data/OptimizationResult초기.xlsx'

# Excel 파일의 시트 데이터를 읽기
cost_data = pd.read_excel(input_file_path, sheet_name='Cost')
distance_data = pd.read_excel(input_file_path, sheet_name='Distance')
city_data = pd.read_excel(input_file_path, sheet_name='City')

# 후보창고 리스트
warehouses = cost_data['WH'].tolist()

# 소비도시 리스트
cities = city_data['City'].tolist()

# 거리 딕셔너리
distance_dict = {}
for _, row in distance_data.iterrows():
    key = (row['FromCity'], row['ToCity'])
    distance_dict[key] = row['Distance']

# 비용 딕셔너리
inbound_rate = dict(zip(cost_data['WH'], cost_data['InboundRate']))
truck_rate = dict(zip(cost_data['WH'], cost_data['TruckUnitRate']))
wh_unit_rate = dict(zip(cost_data['WH'], cost_data['WHUnitRate']))
cargo = dict(zip(city_data['City'], city_data['Cargo']))

# 상수
inventory_turnover = 12
sqrt_factor = math.sqrt(3)

# 모델 정의
model = LpProblem("Warehouse_Optimization", LpMinimize)
warehouse_vars = LpVariable.dicts("Use_WH", warehouses, 0, 1, LpBinary)
city_warehouse_vars = LpVariable.dicts("City_WH", [(city, wh) for city in cities for wh in warehouses], 0, 1, LpBinary)

# 제약조건
for city in cities:
    model += lpSum(city_warehouse_vars[(city, wh)] for wh in warehouses) == 1
for city in cities:
    for wh in warehouses:
        model += city_warehouse_vars[(city, wh)] <= warehouse_vars[wh]
model += lpSum(warehouse_vars[wh] for wh in warehouses) == 3

# 목적함수
total_cost = 0
for city in cities:
    for wh in warehouses:
        distance_key = (wh, city)
        if distance_key in distance_dict:
            distance = distance_dict[distance_key]
        else:
            if wh == city:
                distance = 16
            else:
                continue

        cargo_demand = cargo[city]
        transport_cost = cargo_demand * distance * truck_rate[wh]
        inbound_cost = cargo_demand * inbound_rate[wh]
        warehouse_cost = (cargo_demand / inventory_turnover) * wh_unit_rate[wh] * 12 * sqrt_factor
        total_cost += city_warehouse_vars[(city, wh)] * (transport_cost + inbound_cost + warehouse_cost)

model += total_cost
model.solve()

# 결과 저장
selected_warehouses = [wh for wh in warehouses if warehouse_vars[wh].varValue == 1]
wb = Workbook()

# City Sheet
city_ws = wb.active
city_ws.title = 'City'
city_ws.append(['소비도시', '물동량', '총 물류비', '선택된 창고', '운송비', '수입물류비', '창고비'])

for city in cities:
    for wh in selected_warehouses:
        if city_warehouse_vars[(city, wh)].varValue == 1:
            distance = distance_dict.get((wh, city), 16 if wh == city else None)
            cargo_demand = cargo[city]
            transport_cost = cargo_demand * distance * truck_rate[wh]
            inbound_cost = cargo_demand * inbound_rate[wh]
            warehouse_cost = (cargo_demand / inventory_turnover) * wh_unit_rate[wh] * 12 * sqrt_factor
            total_city_cost = transport_cost + inbound_cost + warehouse_cost

            city_ws.append([city, cargo_demand, total_city_cost, wh, transport_cost, inbound_cost, warehouse_cost])

# WH Sheet
wh_ws = wb.create_sheet('WH')
wh_ws.append(['창고', '수요박스 개수', '운송비', '수입물류비', '창고비', '총 물류비'])

wh_results = []
for wh in selected_warehouses:
    total_transport_cost = 0
    total_inbound_cost = 0
    total_warehouse_cost = 0
    total_cargo = 0

    for city in cities:
        if city_warehouse_vars[(city, wh)].varValue == 1:
            distance = distance_dict.get((wh, city), 16 if wh == city else None)
            cargo_demand = cargo[city]
            total_cargo += cargo_demand
            total_transport_cost += cargo_demand * distance * truck_rate[wh]
            total_inbound_cost += cargo_demand * inbound_rate[wh]
            total_warehouse_cost += (cargo_demand / inventory_turnover) * wh_unit_rate[wh] * 12 * sqrt_factor

    total_wh_cost = total_transport_cost + total_inbound_cost + total_warehouse_cost
    wh_ws.append([wh, total_cargo, total_transport_cost, total_inbound_cost, total_warehouse_cost, total_wh_cost])
    wh_results.append([wh, total_cargo, total_transport_cost, total_inbound_cost, total_warehouse_cost, total_wh_cost])

# Summary Sheet
summary_ws = wb.create_sheet('Summary')
summary_ws.append(['항목', '총 비용'])

total_transport_cost = 0
total_inbound_cost = 0
total_warehouse_cost = 0

for wh in selected_warehouses:
    for city in cities:
        if city_warehouse_vars[(city, wh)].varValue == 1:
            distance = distance_dict.get((wh, city), 16 if wh == city else None)
            cargo_demand = cargo[city]
            total_transport_cost += cargo_demand * distance * truck_rate[wh]
            total_inbound_cost += cargo_demand * inbound_rate[wh]
            total_warehouse_cost += (cargo_demand / inventory_turnover) * wh_unit_rate[wh] * 12 * sqrt_factor

total_summary_cost = total_transport_cost + total_inbound_cost + total_warehouse_cost
summary_ws.append(['운송비', total_transport_cost])
summary_ws.append(['수입물류비', total_inbound_cost])
summary_ws.append(['창고비', total_warehouse_cost])
summary_ws.append(['총 물류비', total_summary_cost])

# 저장
wb.save(output_file_path)

# 출력: Summary Sheet
summary_df = pd.DataFrame({
    '항목': ['운송비', '수입물류비', '창고비', '총 물류비'],
    '총 비용': [total_transport_cost, total_inbound_cost, total_warehouse_cost, total_summary_cost]
})
print("Summary Sheet:")
print(summary_df)

# 출력: WH Sheet
wh_df = pd.DataFrame(wh_results, columns=['창고', '수요박스 개수', '운송비', '수입물류비', '창고비', '총 물류비'])
print("\nWH Sheet:")
print(wh_df)

# 출력: New YorkNY
new_york_results = []
for wh in selected_warehouses:
    if city_warehouse_vars[('New YorkNY', wh)].varValue == 1:
        distance = distance_dict.get((wh, 'New YorkNY'), 16 if wh == 'New YorkNY' else None)
        cargo_demand = cargo['New YorkNY']
        transport_cost = cargo_demand * distance * truck_rate[wh]
        inbound_cost = cargo_demand * inbound_rate[wh]
        warehouse_cost = (cargo_demand / inventory_turnover) * wh_unit_rate[wh] * 12 * sqrt_factor
        total_cost = transport_cost + inbound_cost + warehouse_cost
        new_york_results.append([wh, transport_cost, inbound_cost, warehouse_cost, total_cost])

new_york_df = pd.DataFrame(new_york_results, columns=['창고', '운송비', '수입물류비', '창고비', '총 물류비'])
print("\nNew YorkNY에 대한 최종 선택된 창고의 비용:")
print(new_york_df)


Summary Sheet:
      항목          총 비용
0    운송비  9.627102e+06
1  수입물류비  3.465766e+07
2    창고비  5.862423e+06
3  총 물류비  5.014719e+07

WH Sheet:
              창고  수요박스 개수           운송비         수입물류비           창고비  \
0  Los AngelesCA  3100623  3.830358e+06  7.751558e+06  2.631890e+06   
1      HoustonTX  2407766  2.536167e+06  1.039717e+07  1.381665e+06   
2     ColumbusOH  3228414  3.260576e+06  1.650894e+07  1.848867e+06   

          총 물류비  
0  1.421381e+07  
1  1.431500e+07  
2  2.161838e+07  

New YorkNY에 대한 최종 선택된 창고의 비용:
           창고           운송비         수입물류비           창고비         총 물류비
0  ColumbusOH  1.083639e+06  4.222447e+06  472879.94053  5.778966e+06


# **(수정)3개창고최적화재고증가반영_ChatGPT요청문.txt**
- ChatGPT가 작성한 요청문으로 생성한 파이이썬 코드
- BIP 모델

In [ ]:
# 필요한 라이브러리 설치
!pip install pulp openpyxl

import pandas as pd
import numpy as np
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpBinary
import warnings
from openpyxl import Workbook

# 파일 경로
input_file = '/content/sample_data/NW_MasterData.xlsx'
output_file = '/content/sample_data/OptimizationResult수정BIP.xlsx'

# 데이터 로드
city_df = pd.read_excel(input_file, sheet_name='City')
cost_df = pd.read_excel(input_file, sheet_name='Cost')
distance_df = pd.read_excel(input_file, sheet_name='Distance')

# 상수 정의
inventory_turnover = 12
fixed_distance = 16
num_selected_wh = 3

# 도시 및 창고 목록
cities = city_df['City'].tolist()
warehouses = cost_df['WH'].tolist()

# 거리 딕셔너리 생성
distance_dict = {(row['FromCity'], row['ToCity']): row['Distance'] for _, row in distance_df.iterrows()}

# 변수 초기화
transport_cost = {}
inbound_cost = {}
storage_cost = {}

for city_row in city_df.itertuples():
    city = city_row.City
    demand = city_row.Cargo
    for wh_row in cost_df.itertuples():
        wh = wh_row.WH
        dist = distance_dict.get((wh, city), fixed_distance if wh == city else None)
        if dist is None:
            warnings.warn(f"Distance missing: {wh} to {city}")
            continue

        t_cost = demand * dist * wh_row.TruckUnitRate
        i_cost = demand * wh_row.InboundRate
        s_cost = (demand / inventory_turnover) * wh_row.WHUnitRate * 12 * np.sqrt(num_selected_wh)

        transport_cost[(city, wh)] = t_cost
        inbound_cost[(city, wh)] = i_cost
        storage_cost[(city, wh)] = s_cost

# 모델 정의
model = LpProblem("Warehouse_Optimization", LpMinimize)

# 변수 정의
x = LpVariable.dicts("assign", (cities, warehouses), cat=LpBinary)
y = LpVariable.dicts("select", warehouses, cat=LpBinary)

# 목적함수 정의
model += lpSum([
    (transport_cost[(c, w)] + inbound_cost[(c, w)] + storage_cost[(c, w)]) * x[c][w]
    for c in cities for w in warehouses if (c, w) in transport_cost
])

# 제약조건
# 각 도시는 하나의 창고에만 할당
for c in cities:
    model += lpSum([x[c][w] for w in warehouses if (c, w) in transport_cost]) == 1

# 창고는 정확히 3개 선택
model += lpSum([y[w] for w in warehouses]) == num_selected_wh

# 도시는 선택된 창고에만 할당 가능
for c in cities:
    for w in warehouses:
        if (c, w) in transport_cost:
            model += x[c][w] <= y[w]

# 최적화 수행
model.solve()

# 결과 저장용 리스트 초기화
city_results = []
wh_summary = {w: {'Cargo': 0, 'Transport': 0, 'Inbound': 0, 'Storage': 0} for w in warehouses}

# 도시별 결과 정리
for c in cities:
    for w in warehouses:
        if (c, w) in transport_cost and x[c][w].varValue == 1:
            demand = city_df[city_df['City'] == c]['Cargo'].values[0]
            t = transport_cost[(c, w)]
            i = inbound_cost[(c, w)]
            s = storage_cost[(c, w)]
            total = t + i + s
            city_results.append([c, demand, total, w, t, i, s])

            wh_summary[w]['Cargo'] += demand
            wh_summary[w]['Transport'] += t
            wh_summary[w]['Inbound'] += i
            wh_summary[w]['Storage'] += s
            break

# 시트별 데이터프레임 생성
city_df_out = pd.DataFrame(city_results, columns=['City', 'Cargo', 'TotalCost', 'Warehouse', 'TransportCost', 'InboundCost', 'StorageCost'])
wh_df_out = pd.DataFrame([
    [w, v['Cargo'], v['Transport'], v['Inbound'], v['Storage'], v['Transport'] + v['Inbound'] + v['Storage']]
    for w, v in wh_summary.items() if y[w].varValue == 1
], columns=['Warehouse', 'TotalCargo', 'TransportCost', 'InboundCost', 'StorageCost', 'TotalCost'])

summary_df = pd.DataFrame({
    '항목': ['운송비', '수입물류비', '창고비', '총 물류비'],
    '합계': [
        city_df_out['TransportCost'].sum(),
        city_df_out['InboundCost'].sum(),
        city_df_out['StorageCost'].sum(),
        city_df_out['TotalCost'].sum()
    ]
})

# 결과 출력 (표 형태)
from IPython.display import display
print("\n[Summary Sheet]")
display(summary_df)

print("\n[WH Sheet]")
display(wh_df_out)

print("\n[City Sheet - New YorkNY]")
display(city_df_out[city_df_out['City'] == 'New YorkNY'][['TransportCost', 'InboundCost', 'StorageCost', 'TotalCost']])

# 결과 엑셀 저장
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    city_df_out.to_excel(writer, sheet_name='City', index=False)
    wh_df_out.to_excel(writer, sheet_name='WH', index=False)
    summary_df.to_excel(writer, sheet_name='Summary', index=False)



[Summary Sheet]


,항목,합계
0,운송비,9.627102e+06
1,수입물류비,3.465766e+07
2,창고비,5.862423e+06
3,총 물류비,5.014719e+07



[WH Sheet]


,Warehouse,TotalCargo,TransportCost,InboundCost,StorageCost,TotalCost
0,Los AngelesCA,3100623,3.830358e+06,7.751558e+06,2.631890e+06,1.421381e+07
1,HoustonTX,2407766,2.536167e+06,1.039717e+07,1.381665e+06,1.431500e+07
2,ColumbusOH,3228414,3.260576e+06,1.650894e+07,1.848867e+06,2.161838e+07



[City Sheet - New YorkNY]


,TransportCost,InboundCost,StorageCost,TotalCost
0,1.083639e+06,4.222447e+06,472879.94053,5.778966e+06
